# Trabajo Practico Nº1 - Parte 3: Comunicación y Sincronismo

## Información General

* **Universidad Nacional de la Matanza**
* Materia: Programación Concurrente
* Cuatrimestre: Segundo Cuatrimestre, Año 2024
* Profesores:
    * ADAGIO, MATIAS EZEQUIEL
    * CARNUCCIO, ESTEBAN
    * HIRSCHFELDT, DARIO
    * PALOMO, MAXIMO FACUNDO
    * VOLKER MARIANO LEONARDO
* Grupo: M4


* Integrantes:
    * ANTONIOLI, IVÁN OSCAR
    * DI NICCO, LUIS DEMETRIO
    * SANDOVAL VASQUEZ, JUAN LEANDRO
    * TIGANI MARTIN SEBASTIAN
    * VILLCA, LUIS ALBERTO

## Consigna

**Fecha de entrega**: 25/09/2024

**Forma de entrega**:
Se debe generar un informe que contenga los siguientes puntos:

*   **Carátula**: Con los integrantes del grupo.
*   **Link a un repositorio de GitHub**: En donde se encuentre el archivo del cuaderno de Colab generado para este Trabajo Práctico. Este archivo debe ser con la extension ipynb. También se debe subir a GitHub el código fuente.
*   **Conclusiones**: En esta sección se debe describir las dificultadas que encontraron al realizar el trabajo práctico.

Entregar el informe por plataforma de MIeL. Este debe ser en formato .pdf, con nombre TP1_Parte3_NumeroDelGrupo.pdf.

**Enunciado**: En una reserva natural sudafricana existe un profundo cañón, los babuinos del lugar utilizan una cuerda atada en ambos extremos para cruzar de un lado al otro. La cuerda es lo suficientemente fuerte como para soportar a **cinco (5) babuinos** colgados al mismo tiempo, de superar esa cantidad la cuerda se romperá. Por razones desconocidas si dos babuinos que van en direcciones opuestas se encuentran en medio del cruce, estos se ponen a pelear hasta que alguno cae al vacío.

Desarrolle un programa utilizando algún método de comunicación y/o sincronización que represente el cruce de los babuinos garantizando que ninguno muera.

## Criterios a tener en cuenta

* [Coding Standard](https://google.github.io/styleguide/)
* Ajustes específicos (Ej. llaves en la misma columna)
* Espacios en lugar de tabulaciones (2 espacios)
* Métodos/funciones con menos de 15 líneas.
* Patrones de diseño
* No debe haber números mágicos, uso de constantes descriptivas.

## Resolución sin intercalar direcciones

**Suposiciones**:

Debido a que en la cosnigna no se aclara como definir la cantidad de babuinos que parten de cada lado, ni si solo realizan un viaje o pueden hacer multiples viajes decidimos resolver este ejercicio parte de las siguientes suposiciones.

*   **Cantidad Babuinos De cada Lado**: El programa recibe por parametro la cantidad total de babuinos. En caso de ser una cantidad par, la mitad estarán del lado izquierdo y la otra mitad del lado derecho. En caso de ser una cantidad impar, un lado tendra un babuino mas que el otro.
*   **Cantidad de veces que cruzan**: Vamos a considerar que todos los babuinos solo cruzan una sola vez por la cuerda.
*  **Sentido de Cruce:** Los babuinos empiezan cruzando desde la izquierda a la derecha y cuando terminan los de ese lado comienzan a cruzar de derecha a izquierda.


Para resolver este ejercicio no consideramos necesario utilizar un mecanismo de comunicación.

### Código

In [ ]:
%%writefile reserva_natural.py
import sys
import threading
import time

CANTIDAD_MAXIMA_SOGA = 5
IZQUIERDA = 1
DERECHA = 2
TIEMPO_ESPERA = 1
TIEMPO_PARA_CRUZAR = 1

mutex_region_critica = threading.Lock()
mutex_cuerda = threading.Lock()

cantidad_babuinos = 0
lado_cuerda = 1

def cambiar_de_lado(direccion):
  global lado_cuerda, cantidad_babuinos,cantidad_babuinos_izquierda

  while True:
    if direccion == lado_cuerda:
      with mutex_region_critica:
        if cantidad_babuinos < CANTIDAD_MAXIMA_SOGA:
          cantidad_babuinos += 1
          print(f"Viajando hacia la {'derecha' if IZQUIERDA+direccion%DERECHA == 1 else 'izquierda'}, "
                f"cantidad de babuinos en cuerda: {cantidad_babuinos}")
          sys.stdout.flush()
        else:
          continue

      time.sleep(TIEMPO_PARA_CRUZAR)

      with mutex_region_critica:
        cantidad_babuinos -= 1
        print(f"babuino llegó a la {'derecha' if IZQUIERDA+direccion%DERECHA == 1 else 'izquierda'}, "
        f"cantidad de babuinos en cuerda: {cantidad_babuinos}")
        sys.stdout.flush()

        if cantidad_babuinos == 0:
          print("No hay más babuinos en la cuerda, se libera.")
          sys.stdout.flush()
        if (direccion == 1):
          cantidad_babuinos_izquierda-=1
        return
    else:
      if mutex_cuerda.acquire(blocking=False):
        while( cantidad_babuinos_izquierda > 0):
          time.sleep(TIEMPO_ESPERA)
        print("-------------------------------------------------------------------------------------")
        print(f"Cambiando la dirección de la cuerda hacia la derecha")
        print("-------------------------------------------------------------------------------------")
        sys.stdout.flush()
        lado_cuerda=direccion
        mutex_cuerda.release()


def main():
  global cantidad_babuinos_izquierda

  if len(sys.argv) < 2:
    sys.exit("Se debe indicar la cantidad de monos")
    sys.stdout.flush()
    return
  cantidad_babuinos_a_generar = int(sys.argv[1])
  if cantidad_babuinos_a_generar < 0:
    sys.exit("La cantidad de babuinos no puede ser negativa")
    sys.stdout.flush()

  threads = []
  cantidad_babuinos_izquierda=cantidad_babuinos_a_generar/2

  lado = IZQUIERDA
  for i in range(cantidad_babuinos_a_generar):
    babuino = threading.Thread(target=cambiar_de_lado, args=(lado,))
    threads.append(babuino)
    lado = IZQUIERDA + lado % DERECHA

  for thread in threads:
    thread.start()

  for thread in threads:
    thread.join()


if __name__ == "__main__":
  main()

Writing reserva_natural.py


### Ejecución

Ejecucion de ejemplo, 20 babuinos

In [ ]:
!python reserva_natural.py 20

Viajando hacia la izquierda, cantidad de babuinos en cuerda: 1
Viajando hacia la izquierda, cantidad de babuinos en cuerda: 2
Viajando hacia la izquierda, cantidad de babuinos en cuerda: 3
Viajando hacia la izquierda, cantidad de babuinos en cuerda: 4
Viajando hacia la izquierda, cantidad de babuinos en cuerda: 5
babuino llegó a la izquierda, cantidad de babuinos en cuerda: 4
Viajando hacia la izquierda, cantidad de babuinos en cuerda: 5
babuino llegó a la izquierda, cantidad de babuinos en cuerda: 4
Viajando hacia la izquierda, cantidad de babuinos en cuerda: 5
babuino llegó a la izquierda, cantidad de babuinos en cuerda: 4
Viajando hacia la izquierda, cantidad de babuinos en cuerda: 5
babuino llegó a la izquierda, cantidad de babuinos en cuerda: 4
babuino llegó a la izquierda, cantidad de babuinos en cuerda: 3
Viajando hacia la izquierda, cantidad de babuinos en cuerda: 4
Viajando hacia la izquierda, cantidad de babuinos en cuerda: 5
babuino llegó a la izquierda, cantidad de babuinos

Ejecucion en segundo plano para visualizar los hilos

In [ ]:
!chmod 700 ./reserva_natural.py
!nohup python reserva_natural.py 4 1> salida_python.txt 2>/dev/null &

PID del proceso principal

***Nota***: Seleccionar el PID que sigue el siguiente formato:

root       XXXX  0.0  0.0   6616  2260 ?        S    23:38   0:00 grep reserva_natural

Reemplazar el PID de la celda de abajo por el valor que esta en la posicion "XXXX"

In [ ]:
!ps -aux | grep reserva_natural

root        1433 94.0  0.0 312572  9832 ?        Sl   16:51   0:11 python3 reserva_natural.py 4
root        1488  0.0  0.0   6616  2264 ?        S    16:51   0:00 grep reserva_natural


Hilos generados

***Nota***: Completar el PID con el valor obtenido en la celda de arriba

In [ ]:
!ps -L -p 1433

    PID     LWP TTY          TIME CMD
   1433    1433 ?        00:00:00 python3
   1433    1435 ?        00:00:00 python3
   1433    1436 ?        00:00:00 python3
   1433    1437 ?        00:00:00 python3
   1433    1438 ?        00:00:18 python3


Salida

In [ ]:
!cat salida_python.txt

Viajando hacia la izquierda, cantidad de babuinos en cuerda: 1
Viajando hacia la izquierda, cantidad de babuinos en cuerda: 2
babuino llegó a la izquierda, cantidad de babuinos en cuerda: 1
babuino llegó a la izquierda, cantidad de babuinos en cuerda: 0
No hay más babuinos en la cuerda, se libera.
-------------------------------------------------------------------------------------
Cambiando la dirección de la cuerda hacia la derecha
-------------------------------------------------------------------------------------
Viajando hacia la derecha, cantidad de babuinos en cuerda: 1
Viajando hacia la derecha, cantidad de babuinos en cuerda: 2
babuino llegó a la derecha, cantidad de babuinos en cuerda: 1
babuino llegó a la derecha, cantidad de babuinos en cuerda: 0
No hay más babuinos en la cuerda, se libera.




## Version intercalando las direcciones


**Suposiciones**:

Debido a que en la cosnigna no se aclara como definir la cantidad de babuinos que parten de cada lado, ni si solo realizan un viaje o pueden hacer multiples viajes decidimos resolver este ejercicio parte de las siguientes suposiciones.

*   **Cantidad Babuinos De cada Lado**: El programa recibe por parametro la cantidad total de babuinos. En esta version la cantidad de babuinos de cada lado es aleatoria.
*   **Cantidad de veces que cruzan**: Vamos a considerar que todos los babuinos solo cruzan una sola vez por la cuerda.


En este caso si tuvimos que usar un mecanismo de comunicacion, utilizamos notify() y notify_all() para permitir la comunicación entre los babuinos que están esperando en cada lado de la cuerda. Esto les indica cuándo pueden intentar cruzar, basándose en la disponibilidad de la cuerda.

### Codigo

In [ ]:
%%writefile reserva_natural_2.py
import threading
import time
import random
import argparse

MINIMO_GRUPO_BABUINOS = 5
PROPORCION_GRUPO_BABUINOS = 4
MAX_BABUINOS_CUERDA = 5
TIEMPO_EN_CUERDA = 1

babuinos_este = 0
babuinos_oeste = 0
babuinos_esperando_este = 0
babuinos_esperando_oeste = 0
prioridad_direccion = 'ninguna'
babuinos_cruzaron_direccion_actual = 0
babuinos_total = 0
babuinos_contador = 0
babuinos_faltan_este = 0
babuinos_faltan_oeste = 0

semaforo_cuerda = threading.Semaphore(MAX_BABUINOS_CUERDA)

condicion_este = threading.Condition()
condicion_oeste = threading.Condition()

def entra_babuino_este():
  global babuinos_esperando_este, babuinos_este, babuinos_faltan_este
  global babuinos_cruzaron_direccion_actual, prioridad_direccion
  with condicion_este:
    babuinos_esperando_este += 1
    while no_puede_cruzar_este():
      condicion_este.wait()
    babuinos_esperando_este -= 1
    semaforo_cuerda.acquire()
    babuinos_este += 1
    prioridad_direccion = 'este'
    babuinos_cruzaron_direccion_actual += 1
    babuinos_faltan_este -= 1
    print(f"Babuino cruzando hacia el este. Total en cuerda: {MAX_BABUINOS_CUERDA - semaforo_cuerda._value}")
    print(f"Babuinos que faltan cruzar al este: {babuinos_faltan_este}")

def entra_babuino_oeste():
  global babuinos_esperando_oeste, babuinos_oeste, babuinos_faltan_oeste
  global babuinos_cruzaron_direccion_actual, prioridad_direccion
  with condicion_oeste:
    babuinos_esperando_oeste += 1
    while no_puede_cruzar_oeste():
      condicion_oeste.wait()
    babuinos_esperando_oeste -= 1
    semaforo_cuerda.acquire()
    babuinos_oeste += 1
    prioridad_direccion = 'oeste'
    babuinos_cruzaron_direccion_actual += 1
    babuinos_faltan_oeste -= 1
    print(f"Babuino cruzando hacia el oeste. Total en cuerda: {MAX_BABUINOS_CUERDA - semaforo_cuerda._value}")
    print(f"Babuinos que faltan cruzar al oeste: {babuinos_faltan_oeste}")

def no_puede_cruzar_este():
  if babuinos_faltan_oeste == 0:
    return (babuinos_oeste > 0 or semaforo_cuerda._value == 0 or
            (prioridad_direccion == 'oeste' and babuinos_oeste > 0))
  else:
    return (babuinos_oeste > 0 or semaforo_cuerda._value == 0 or
            (prioridad_direccion == 'oeste' and babuinos_oeste > 0) or
            (babuinos_cruzaron_direccion_actual >= max_babuinos_grupo and babuinos_esperando_oeste > 0))

def no_puede_cruzar_oeste():
  if babuinos_faltan_este == 0:
    return (babuinos_este > 0 or semaforo_cuerda._value == 0 or
            (prioridad_direccion == 'este' and babuinos_este > 0))
  else:
    return (babuinos_este > 0 or semaforo_cuerda._value == 0 or
            (prioridad_direccion == 'este' and babuinos_este > 0) or
            (babuinos_cruzaron_direccion_actual >= max_babuinos_grupo and babuinos_esperando_este > 0))

def sale_babuino_este():
  global babuinos_este, babuinos_contador
  with condicion_este:
    babuinos_este -= 1
    babuinos_contador += 1
    semaforo_cuerda.release()
    print(f"Babuino terminó de cruzar hacia el este. Total en cuerda: {MAX_BABUINOS_CUERDA - semaforo_cuerda._value}")

    if (MAX_BABUINOS_CUERDA - semaforo_cuerda._value) == 0 and babuinos_contador < babuinos_total:
      if babuinos_faltan_oeste > 0:
        print("------------------------------------------------------------------------------------->>>")
        print(f"Cambiando la dirección de la cuerda hacia el oeste")
        print(f"Babuinos que faltan cruzar al este: {babuinos_faltan_este}, al oeste: {babuinos_faltan_oeste}")
        print("<<<-------------------------------------------------------------------------------------")
        reiniciar_direccion()
        with condicion_oeste:
          condicion_oeste.notify_all()
      else:
        reiniciar_direccion()
        with condicion_este:
          condicion_este.notify_all()

def sale_babuino_oeste():
  global babuinos_oeste, babuinos_contador
  with condicion_oeste:
    babuinos_oeste -= 1
    babuinos_contador += 1
    semaforo_cuerda.release()
    print(f"Babuino terminó de cruzar hacia el oeste. Total en cuerda: {MAX_BABUINOS_CUERDA - semaforo_cuerda._value}")

    if (MAX_BABUINOS_CUERDA - semaforo_cuerda._value) == 0 and babuinos_contador < babuinos_total:
      if babuinos_faltan_este > 0:
        print("<<<-------------------------------------------------------------------------------------")
        print(f"Cambiando la dirección de la cuerda hacia el este")
        print(f"Babuinos que faltan cruzar al este: {babuinos_faltan_este}, al oeste: {babuinos_faltan_oeste}")
        print("------------------------------------------------------------------------------------->>>")
        reiniciar_direccion()
        with condicion_este:
          condicion_este.notify_all()
      else:
        reiniciar_direccion()
        with condicion_oeste:
          condicion_oeste.notify_all()

def reiniciar_direccion():
  global babuinos_cruzaron_direccion_actual
  babuinos_cruzaron_direccion_actual = 0

def babuino(direccion):
  if direccion == 'este':
    entra_babuino_este()
  else:
    entra_babuino_oeste()
  time.sleep(TIEMPO_EN_CUERDA)
  if direccion == 'este':
    sale_babuino_este()
  else:
    sale_babuino_oeste()

def iniciar_babuinos(cantidad_babuinos):
  global babuinos_total, max_babuinos_grupo, babuinos_faltan_este, babuinos_faltan_oeste
  babuinos_total = cantidad_babuinos
  max_babuinos_grupo = max(cantidad_babuinos // PROPORCION_GRUPO_BABUINOS, MINIMO_GRUPO_BABUINOS)

  babuinos = []
  direcciones_asignadas = []

  for _ in range(cantidad_babuinos):
    direccion = random.choice(['este', 'oeste'])
    direcciones_asignadas.append(direccion)
    if direccion == 'este':
      babuinos_faltan_este += 1
    else:
      babuinos_faltan_oeste += 1

  print(f"Babuinos que faltan cruzar al este: {babuinos_faltan_este}, al oeste: {babuinos_faltan_oeste}")
  print("----------------------------------------------------------------------------------------")

  for direccion in direcciones_asignadas:
    t = threading.Thread(target=babuino, args=(direccion,))
    babuinos.append(t)
    t.start()

  for t in babuinos:
    t.join()

def main():
  parser = argparse.ArgumentParser(description="Simulación de babuinos cruzando la cuerda.")
  parser.add_argument("cantidad_babuinos", type=int, help="Cantidad total de babuinos")
  args = parser.parse_args()

  print("----------------------------------------------------------------------------------------")
  print("Comienza la simulación")
  print("----------------------------------------------------------------------------------------")
  iniciar_babuinos(args.cantidad_babuinos)
  print("----------------------------------------------------------------------------------------")
  print("Finaliza la simulación")
  print("----------------------------------------------------------------------------------------")

if __name__ == "__main__":
  main()

Overwriting reserva_natural_2.py


### Ejecución

Ejecucion de ejemplo, 20 babuinos

In [ ]:
!python reserva_natural_2.py 20

----------------------------------------------------------------------------------------
Comienza la simulacion
----------------------------------------------------------------------------------------
Babuino cruzando hacia el este. Total en cuerda: 1
Babuino cruzando hacia el este. Total en cuerda: 2
Babuino cruzando hacia el este. Total en cuerda: 3
Babuino cruzando hacia el este. Total en cuerda: 4
Babuino cruzando hacia el este. Total en cuerda: 5
Babuino terminó de cruzar hacia el este. Total en cuerda: 4
Babuino terminó de cruzar hacia el este. Total en cuerda: 3
Babuino terminó de cruzar hacia el este. Total en cuerda: 2
Babuino terminó de cruzar hacia el este. Total en cuerda: 1
Babuino terminó de cruzar hacia el este. Total en cuerda: 0
------------------------------------------------------------------------------------->>>
Cambiando la dirección de la cuerda hacia el oeste
<<<-------------------------------------------------------------------------------------
Babuino cruzand

Ejecucion en segundo plano para visualizar los hilos

In [ ]:
!chmod 700 ./reserva_natural_2.py
!nohup python reserva_natural_2.py 4 1> salida_python_2.txt 2>/dev/null &

PID del proceso principal

***Nota***: Seleccionar el PID que sigue el siguiente formato:

root       XXXX  0.0  0.0   6616  2260 ?        S    23:38   0:00 grep reserva_natural_2

Reemplazar el PID de la celda de abajo por el valor que esta en la posicion "XXXX"

In [ ]:
!ps -aux | grep reserva_natural_2

root        4760  0.1  0.0 314348 11512 ?        Sl   15:55   0:00 python3 reserva_natural_2.py 4
root        4894  0.0  0.0   6616  2364 ?        S    15:56   0:00 grep reserva_natural_2


Hilos generados

***Nota***: Completar el PID con el valor obtenido en la celda de arriba

In [ ]:
!ps -L -p 4760

    PID     LWP TTY          TIME CMD
   4760    4760 ?        00:00:00 python3
   4760    4762 ?        00:00:00 python3
   4760    4763 ?        00:00:00 python3
   4760    4764 ?        00:00:00 python3
   4760    4765 ?        00:00:00 python3


Salida

In [ ]:
!cat salida_python_2.txt

----------------------------------------------------------------------------------------
Comienza la simulación
----------------------------------------------------------------------------------------
Babuinos que faltan cruzar al este: 4, al oeste: 0
----------------------------------------------------------------------------------------
Babuino cruzando hacia el este. Total en cuerda: 1
Babuinos que faltan cruzar al este: 3
Babuino cruzando hacia el este. Total en cuerda: 2
Babuinos que faltan cruzar al este: 2
Babuino cruzando hacia el este. Total en cuerda: 3
Babuinos que faltan cruzar al este: 1
Babuino cruzando hacia el este. Total en cuerda: 4
Babuinos que faltan cruzar al este: 0
Babuino terminó de cruzar hacia el este. Total en cuerda: 3
Babuino terminó de cruzar hacia el este. Total en cuerda: 2
Babuino terminó de cruzar hacia el este. Total en cuerda: 1
Babuino terminó de cruzar hacia el este. Total en cuerda: 0
---------------------------------------------------------------

Version sin intercar las direcciones